In [1]:
import chess

In [2]:
import chess.pgn

In [3]:
import numpy as np

In [4]:
pgn = open("dataset1.pgn")
game = chess.pgn.read_game(pgn)
game.headers

Headers(Event='Rated Classical game', Site='https://lichess.org/j1dkb5dw', Date='????.??.??', Round='?', White='BFG9k', Black='mamalak', Result='1-0', BlackElo='1403', BlackRatingDiff='-8', ECO='C00', Opening='French Defense: Normal Variation', Termination='Normal', TimeControl='600+8', UTCDate='2012.12.31', UTCTime='23:01:03', WhiteElo='1639', WhiteRatingDiff='+5')

In [5]:
pieces = ['k', 'q', 'r', 'b', 'n', 'p', 'K', 'Q', 'R', 'B', 'N', 'P']
alpha = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
nums = ['1', '2', '3', '4', '5', '6', '7', '8']

In [6]:
def move_to_output(move):
    output = np.zeros(32)
    output[alpha.index(move[0])] = 1
    output[8 + nums.index(move[1])] = 1
    output[16 + alpha.index(move[2])] = 1
    output[24 + nums.index(move[3])] = 1
    return output

In [7]:
def get_moves_array(file_path):
    pgn = open(file_path)
    moves = []
    cnt = 0
    while cnt < 20000:
        # for each game open its moves
        game = chess.pgn.read_game(pgn)
        if game.headers["WhiteElo"] == '?' or game.headers["BlackElo"] == '?':
            continue
        if int(game.headers["WhiteElo"]) > 1700 or int(game.headers["WhiteElo"]) < 1300 or int(game.headers["BlackElo"]) > 1700 or int(game.headers["BlackElo"]) < 1300:
            continue
        # print(game.headers["WhiteElo"], game.headers["BlackElo"])
        game_moves = []
        for move in game.mainline_moves():
            game_moves.append(str(move))

        moves.append(game_moves)
        cnt += 1
        if cnt % 1000 == 0:
            print(f"{cnt} done! ", end = ' ')

    return moves

In [8]:
moves = get_moves_array("dataset1.pgn")

1000 done!  2000 done!  3000 done!  4000 done!  5000 done!  6000 done!  7000 done!  8000 done!  9000 done!  10000 done!  11000 done!  12000 done!  13000 done!  14000 done!  15000 done!  16000 done!  17000 done!  18000 done!  19000 done!  20000 done!  

In [9]:
b = chess.Board()
fen = b.fen()
fen_data = fen.split(' ')[0]
# print(fen_data)
fen_array = fen_data.split('/')
# print(fen_array)
board = np.zeros((12, 8, 8))
i = j = 0
for row in fen_array:
    for ch in row:
        if ch in pieces:
            board[pieces.index(ch)][i][j] = 1
            # print(ch, pieces.index(ch))
            j += 1
        else:
            j += int(ch)
    i += 1
    j = 0
# print(pieces)
# init_board = board.ravel()
init_board = board
np.save('board', init_board)

In [10]:
def push_move_to_board(board, move):
    alpha1 = alpha.index(move[0])
    alpha2 = alpha.index(move[2])
    num1 = 7 - nums.index(move[1])
    num2 = 7 - nums.index(move[3])
    p = 0
    for i in range(12):
        # if board[64*i+num1*8+alpha1] == 1:
        if board[i, num1, alpha1] == 1:
            p = i
            break
    # print(p)
    # board[64*p+num1*8+alpha1] = 0
    board[i, num1, alpha1] = 0
    for i in range(12):
        # if board[64*i+num2*8+alpha2] == 1:
        if board[i, num2, alpha2] == 1:
            # board[64*i+num2*8+alpha2] = 0
            board[i, num2, alpha2] = 0
            break
    # board[64*p+num2*8+alpha2] = 1
    board[p, num2, alpha2] = 1
    return board

In [16]:
def add_games(moves):
    X = []
    y = []
    cnt = 0
    # df = pd.DataFrame(columns = np.arange(32))
    for move_list in moves:
        board = chess.Board()
        l = 0
        game_board = init_board.copy()
        capt, cast = 0, 0
        for move in move_list:
            # get its position and move data
            # print(board.fen())
            pos_data = game_board.ravel()
            xtras = np.zeros(10)
            # print(pos_data)
            # opposite pieces
            b, kn, r, p, q = '', '', '', '', ''
            if l % 2:
#                 pos_data.append(0)
                xtras[0] = 0
                q = bin(board.pieces_mask(chess.QUEEN, 0))
                b = bin(board.pieces_mask(chess.BISHOP, 0))
                kn = bin(board.pieces_mask(chess.KNIGHT, 0))
                r = bin(board.pieces_mask(chess.ROOK, 0))
                p = bin(board.pieces_mask(chess.PAWN, 0))
            else:
#                 pos_data.append(1)
                xtras[0] = 1
                q = bin(board.pieces_mask(chess.QUEEN, 1))
                b = bin(board.pieces_mask(chess.BISHOP, 1))
                kn = bin(board.pieces_mask(chess.KNIGHT, 1))
                r = bin(board.pieces_mask(chess.ROOK, 1))
                p = bin(board.pieces_mask(chess.PAWN, 1))
            # print(pos_data)
            total = 0
            for c in q[2:]:
                total += int(c)
            xtras[1] = total
            
            total = 0
            for c in b[2:]:
                total += int(c)
#             pos_data.append(total/2)
            xtras[2] = total/2

            total = 0
            for c in kn[2:]:
                total += int(c)
#             pos_data.append(total/2)
            xtras[3] = total/2

            total = 0
            for c in r[2:]:
                total += int(c)
#             pos_data.append(total/2)
            xtras[4] = total/2

            total = 0
            for c in p[2:]:
                total += int(c)
#             pos_data.append(total/8)
            xtras[5] = total/8
            
            xtras[6] = board.is_check()
#             pos_data.append(board.is_check())
            
#             try:
            xtras[7:9] = (capt, cast)
            capt = board.is_capture(board.parse_uci(move))
            cast = board.is_castling(board.parse_uci(move))
            
            xtras[9] = len(list(board.legal_moves))/50
            
            pos_data = np.append(pos_data, xtras)
            # xtras[0, 4] = total/4
            # print(pos_data)
            # pos_data = np.append(pos_data, xtras.reshape(1, 8, 8), axis = 0)
            # print(pos_data)
            # if l == 0:
            #     print(pos_data)
            # pos_data = np.zeros((12, 8, 8))
            move_data = move_to_output(move)
            # push the move to board
            board.push_san(move)
            game_board = push_move_to_board(game_board.copy(), str(move))
            # add to the dataframes
            # df.loc[l] = move_data.reshape(768,)
            pos_data = np.array(pos_data)
            X.append(pos_data)
            # X.append(pos_data)
            y.append(move_data)
            l += 1

        cnt += 1
        if cnt % 1000 == 0:
            print(f"{cnt} done! ", end = ' ')
        if cnt % 10000 == 0:
            print("\n")

    return X, y

In [17]:
X_out, y_out = add_games(moves)

1000 done!  2000 done!  3000 done!  4000 done!  5000 done!  6000 done!  7000 done!  8000 done!  9000 done!  10000 done!  

11000 done!  12000 done!  13000 done!  14000 done!  15000 done!  16000 done!  17000 done!  18000 done!  19000 done!  20000 done!  



In [18]:
X = np.array(X_out)
y = np.array(y_out)
X.shape, y.shape

((1321933, 778), (1321933, 32))

In [17]:
np.save('X', X)
np.save('Y', y)

In [13]:
board = chess.Board()